In [8]:
%run bert.ipynb

attention pattern raw MATCH!!!!!!!!
 SHAPE (2, 12, 3, 3) MEAN: -0.002689 STD: 0.1037 VALS [-0.0166 0.0759 -0.0623 -0.0958 0.04385 0.1269 -0.02935 -0.169 0.04702 0.04498...]
attention MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: 0.004806 STD: 0.1195 VALS [0.1319 -0.01158 -0.05224 0.2031 -0.01778 0.1567 0.01941 -0.01889 0.02216 0.07382...]
bert MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: -0.001554 STD: 0.1736 VALS [-0.08316 -0.09165 -0.03188 -0.03013 0.1001 0.09549 -0.1046 0.07742 0.0424 0.05553...]
bert mlp MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: -0.0001934 STD: 0.1044 VALS [-0.1153 0.1189 -0.0813 0.1021 0.0296 0.06182 0.0341 0.1446 0.2622 -0.08507...]
layer norm MATCH!!!!!!!!
 SHAPE (20, 10) MEAN: -1.431e-08 STD: 1.003 VALS [0.6906 -0.84 1.881 1.711 -0.5116 -0.9577 -0.1387 -0.6943 -0.6741 -0.4662...]
bert MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: -5.381e-09 STD: 1 VALS [0.007132 -0.04372 0.6502 -0.5972 -1.097 0.7267 0.1275 -0.6035 -0.2226 0.2145...]
{'input_ids': [[101, 8667, 117, 146, 1821, 170, 565

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


comparing Berts MATCH!!!!!!!!
 SHAPE (10, 20, 28996) MEAN: -2.796 STD: 2.407 VALS [-5.172 -5.74 -5.461 -5.619 -5.017 -5.26 -5.695 -5.955 -5.663 -5.635...]


In [9]:
import transformers
from tqdm.notebook import tqdm
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")

In [10]:
def make_bert():
    my_bert = Bert(
        vocab_size=28996, hidden_size=768, max_position_embeddings=512, 
        type_vocab_size=2, dropout=0.1, intermediate_size=3072, 
        num_heads=12, num_layers=12
    )
    pretrained_state_dict = pretrained_bert.state_dict()

    #pretrained_keys = pretrained_state_dict.keys()
    #my_keys = my_bert.state_dict().keys()
    #print(set(my_keys) - set(pretrained_keys))

    # we could load these but actually we want to be fine-tuning this stuff in w2d2
    # so let's not take these 
    del pretrained_state_dict["classification_head.weight"]
    del pretrained_state_dict["classification_head.bias"]

    my_bert.load_state_dict(pretrained_state_dict, strict=False)
    return my_bert

In [11]:
def ascii_art_probs(sentence):
    bert = make_bert()
    token_ids = torch.Tensor(tokenizer.encode(sentence)).long().unsqueeze(0)
    mask_idxs = set()
    for idx, token_id in enumerate(token_ids[0]):
        if token_id == 103: # 103 == [MASK]
            mask_idxs.add(idx)
    unnormalized_output = bert(token_ids)
    output = torch.log_softmax(unnormalized_output, dim=-1)
    top_k = torch.topk(output, 5, dim=-1)
    results = []
    for seq_i, seq_top_k in enumerate(top_k.indices[0]):
        if seq_i in mask_idxs:
            results.append(list(zip(tokenizer.convert_ids_to_tokens(seq_top_k), [round(x.item(), 2) for x in top_k.values[0][seq_i].exp()])))
    for l in results:
        print(l)

ascii_art_probs("The fish loves to eat [MASK] and [MASK] and [MASK] and [MASK] and [MASK] and [MASK] and [MASK].")


[('fish', 0.16), ('meat', 0.09), ('eggs', 0.07), ('food', 0.04), ('apples', 0.03)]
[('fish', 0.11), ('eggs', 0.07), ('meat', 0.04), ('fruit', 0.02), ('bread', 0.02)]
[('fish', 0.08), ('meat', 0.06), ('eggs', 0.05), ('bread', 0.02), ('vegetables', 0.02)]
[('fish', 0.05), ('meat', 0.03), ('eggs', 0.02), ('apples', 0.01), ('fruit', 0.01)]
[('fish', 0.06), ('meat', 0.03), ('eggs', 0.03), ('salt', 0.02), ('fruit', 0.02)]
[('fish', 0.06), ('eggs', 0.03), ('meat', 0.03), ('birds', 0.02), ('fruit', 0.02)]
[('fish', 0.04), ('more', 0.03), ('birds', 0.02), ('eggs', 0.01), ('honey', 0.01)]


In [12]:
# class ClassificationBERT():
#     def __init__(self, num_classes):
#         VOCAB_SIZE = 28996
#         DROPOUT=0.1
#         self.bert = Bert(
#             vocab_size=VOCAB_SIZE, hidden_size=768, max_position_embeddings=512, 
#             type_vocab_size=2, dropout=DROPOUT, intermediate_size=3072, 
#             num_heads=12, num_layers=12
#         )
#         self.dropout = torch.nn.Dropout(DROPOUT)
#         # TODO attach this to the last block of BERT as a hidden_size x num_classes linear layer,
#         # not on top of the vocab_size output of BERT
#         self.classification_head = torch.nn.Linear(VOCAB_SIZE, num_classes)

#         pretrained_state_dict = pretrained_bert.state_dict()
#         del pretrained_state_dict["classification_head.weight"]
#         del pretrained_state_dict["classification_head.bias"]
#         self.bert.load_state_dict(pretrained_state_dict)

#     def forward(self, X):
#         return self.classification_head(self.dropout(self.bert(X)))

In [13]:
import torchtext
import random
data_train, data_test = torchtext.datasets.IMDB(root='.data', split=('train', 'test'))
data_train_list = list(data_train)
random.shuffle(data_train_list)

In [14]:

def make_batches(data_train_list, batch_size, max_seq_len):
    batches = []
    for batch_start in range(0, len(data_train_list), batch_size):
        batch_end = min(batch_start + batch_size, len(data_train_list))
        ratings, sentences = [list(tup) for tup in list(zip(*data_train_list[batch_start:batch_end]))]
        tokens = tokenizer(sentences, padding='longest', max_length=max_seq_len, truncation=True)
        ratings01 = torch.tensor([ rating == 'pos' for rating in ratings ], dtype=torch.long)
        tokens_tensor = torch.tensor(tokens.input_ids, dtype=torch.long)
        batches.append((tokens_tensor, ratings01))
    return batches

In [15]:
make_batches(data_train_list[:4], 2, 10)

[(tensor([[ 101,  146, 4417, 1137, 1120, 1655, 2824, 1451, 3017,  102],
          [ 101, 1409, 1103, 6621, 1104, 1109, 4537, 1622, 4499,  102]]),
  tensor([0, 1])),
 (tensor([[  101,   146,  1660,  1142,  2523,   170,   124,  1112,  1122,   102],
          [  101,   120,   115,  6812,   188,  5674, 25614,  1116,   115,   102]]),
  tensor([0, 1]))]

In [16]:
bert = make_bert().cuda()
lossfn = nn.CrossEntropyLoss()
lr = 1e-5
optimizer = torch.optim.Adam(bert.parameters(), lr)
data = make_batches(data_train_list, 128, 512)

In [17]:
def train(model, optimizer, data, lossfn):
    model.train()
    i = 0
    for X, y in tqdm(data):
        optimizer.zero_grad()
        X = X.cuda()
        y_hat = model(X, classification_mode=True)[:,0].softmax(dim=-1)
        loss = lossfn(y_hat, y.cuda())
        loss.backward()
        optimizer.step()
        i += 1
        if i % 30 == 0:
            print(loss)
train(bert, optimizer, data, lossfn)

  0%|          | 0/196 [00:00<?, ?it/s]

tensor(0.6984, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6961, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6908, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6872, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7069, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7012, device='cuda:0', grad_fn=<NllLossBackward0>)
